In [ ]:
from time import time
from pyspark import SparkContext

# test spark with 6 workers
for j in range(1,7):
    sc = SparkContext(master=f'local[{j}]')
    t0 = time()
    for i in range(6):
        sc.parallelize([1,2] * 1000000).reduce(lambda x,y:x+y)
    print(f'{j} executors, time= {time() - t0}')
    sc.stop()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[6]").appName('reddit_wordcloud').getOrCreate()
comments = spark.read.format("csv")\
                     .option("sep", "\t")\
                     .options(inferschema='true',header='true')\
                     .load('/home/ubuntu/cs179g_venv/src/comments_join.tsv')
comments_df = comments.toPandas()

# clean up comments_df data
comments_df = comments_df.rename({'Title' : 'Title', 'Combo': 'Comments'}, axis=1)
comments_df = comments_df.drop('Title', axis=1)
comments_df.Comments = comments_df.Comments.fillna('N/A') # bug with dropping NaNs, so fill with 'N/A'
comments_df = comments_df[comments_df.Comments != 'N/A']  # drop rows with 'N/A'
print(comments_df)

In [ ]:
### wordcloud with words from 'comments_join'
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

comments_df_list = []

# add all rows of short comments to a list
for c in comments_df['Comments']:
    comments_df_list.append(c)

comments_str = ""
for c in comments_df['Comments']:
    comments_str += c + " "

plt.figure(figsize = (20,16)) 
wc = WordCloud(max_words = 10000, width = 2000, height = 1000).generate(" ".join(comments_df.Comments))
plt.imshow(wc, interpolation = 'bilinear')
plt.axis("off")
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

titles_df = pd.read_csv("/home/ubuntu/cs179g_venv/src/all_posts_clean.csv")
titles_df = titles_df['Title']

plt.figure(figsize = (20,16)) 
wc = WordCloud(max_words = 10000, width = 2000, height = 1000).generate(" ".join(titles_df))
plt.imshow(wc, interpolation = 'bilinear')
plt.axis("off")
plt.show()